In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tensorflow as tf
import matplotlib
import os
import matplotlib.pyplot as plt
from datetime import datetime
tf.set_random_seed(77)

mockForecastDictionary = {}
realForecastDictionary = {}

def LearningModuleRunner(rawArrayDatas, processId, forecastDay,dayOrWeekOrMonth):
    #TODO make dayOrWeekOrMonth parameter
    dayOrWeekOrMonth=dayOrWeekOrMonth
    # options:
    # 'day', 'week', 'month'

    feature = 'DayOfWeek_WeekNumber_Month_Season'
    # options:
    # dayOrWeekOrMonth='day': 'DayOfWeek_WeekNumber_Month_Season','DayOfWeek01_WeekNumber_Month_Season'//
    # dayOrWeekOrMonth='week': 'WeekNumber_Month_Season_Year'

#     LoggingManager.PrintLogMessage("LearningManager", "LearningModuleRunner", "start of learning #" + str(processId), DefineManager.LOG_LEVEL_INFO)

    global mockForecastDictionary
    global realForecastDictionary
    mockForcastDay=forecastDay

    ##Make txsForRealForecastLstm   [:]
    ds = rawArrayDatas[0]
    y = list(np.sqrt(rawArrayDatas[1]))
    sales = list(zip(ds, y))
    txsForRealForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastLstm [:-forecastDay]
    ds = rawArrayDatas[0][:-forecastDay]
    y= list(np.sqrt(rawArrayDatas[1][:-forecastDay] ))
    sales = list(zip(ds, y))
    txsForMockForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForRealForecastBayesian [:-forecastDay] & np.log
    ds = rawArrayDatas[0][:-forecastDay]
    # TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y = list(np.log(rawArrayDatas[1][:-forecastDay]))
    sales = list(zip(ds, y))
    txsForRealForecastBayesian = pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastBayseian   [:-(mockForcastDay+forecastDay)] & np.log
    ds = rawArrayDatas[0][:-(mockForcastDay+forecastDay)]
    #TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y= list(np.log(rawArrayDatas[1][:-(mockForcastDay+forecastDay)]))
    sales = list(zip(ds, y))
    txsForMockForecastBayseian =pd.DataFrame(data=sales, columns=['ds', 'y'])

    #testY for algorithm compare has size of (mockForcastDay+forecastDay)  rawArrayDatas[1][-(mockForcastDay+forecastDay):-forecastDay]
    testY= rawArrayDatas[1][-(mockForcastDay+forecastDay):-forecastDay]


    if dayOrWeekOrMonth is 'day':
        ####LSTM_day

        #select feature module
        feature='DayOfWeek_WeekNumber_Month_Season'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay,feature)

      
        ####Bayseian_day

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'day')

        #알고리즘 비교
        nameOfBestAlgorithm= AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출       
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'day')
            
    elif dayOrWeekOrMonth is 'week':
        
        ####LSTM_week

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)


        ####Bayseian_week

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'week')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'week')
    
            
    elif dayOrWeekOrMonth is 'month':
       
        ####LSTM_month

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)    

        ####Bayseian_month

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'month')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'month')

        ####################################################################################BAYSEIAN
    print('LSTM realforecast :',realForecastDictionary['LSTM'])
    print('Bayseian realforecast :',realForecastDictionary['Bayseian'] ) 
    data = rawArrayDatas[1][:-forecastDay] + realForecastDictionary[nameOfBestAlgorithm]
    date= rawArrayDatas[0]
    print("===================================THE END===================================================")
    return realForecastDictionary[nameOfBestAlgorithm]


def LSTM(txs, forecastDay, features):
    tf.reset_default_graph()
    tf.set_random_seed(77)
    # Add basic date related features to the table
    year = lambda x: datetime.strptime(x, "%Y-%m-%d").year
    dayOfWeek = lambda x: datetime.strptime(x, "%Y-%m-%d").weekday()
    month = lambda x: datetime.strptime(x, "%Y-%m-%d").month
    weekNumber = lambda x: datetime.strptime(x, "%Y-%m-%d").strftime('%V')
    txs['year'] = txs['ds'].map(year)
    txs['month'] = txs['ds'].map(month)
    txs['weekNumber'] = txs['ds'].map(weekNumber)
    txs['dayOfWeek'] = txs['ds'].map(dayOfWeek)

    # Add non-basic date related features to the table
    seasons = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0]  # dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d").month - 1)]
    day_of_week01s = [0, 0, 0, 0, 0, 1, 1]
    day_of_week01 = lambda x: day_of_week01s[(datetime.strptime(x, "%Y-%m-%d").weekday())]
    txs['season'] = txs['ds'].map(season)
    txs['dayOfWeek01'] = txs['ds'].map(day_of_week01)

    # Backup originalSales
    originalSales = list(txs['y'])
    sales = list(txs['y'])
    
    #week number는 경계부분에서 약간 오류가 있다.
    if features is 'DayOfWeek_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]
    elif features is 'DayOfWeek01_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek01']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]

    elif features is 'WeekNumber_Month_Season_Year':
        tempxy = [list(txs['weekNumber']), list(txs['month']), list(txs['season']), list(txs['year']), sales]
    
   
    xy = np.array(tempxy).transpose().astype(np.float)
    


    # Backup originalXY for denormalize
    originalXY = np.array(tempxy).transpose().astype(np.float)
    xy = minMaxNormalizer(xy)

    # TRAIN PARAMETERS
    # data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
    data_dim = 5
    # data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
    seq_length = 10
    # output_dim(=forecastDays)만큼의 다음날 y_data를 예측
    output_dim = forecastDay
    # hidden_dim은 정말 임의로 설정
    hidden_dim = 100
    # learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
    learning_rate = 0.001
    iterations=1000
    # Build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
    x = xy
    y = xy[:, [-1]]
    dataX = []
    dataY = []
    for i in range(0, len(y) - seq_length - forecastDay + 1):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length:i + seq_length + forecastDay]
        _y = np.reshape(_y, (forecastDay))
        dataX.append(_x)
        dataY.append(_y)

    train_size = int(len(dataY) - forecastDay)
    # train_size = int(len(dataY) * 0.7)
    test_size = len(dataY) - train_size

    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])

    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, forecastDay])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)

    denormalizedTestY = originalSales[train_size + seq_length:]
    #     denormalizedTestY_feed=np.array([[i] for i in denormalizedTestY])

    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])

    count = 0
    with tf.Session() as sess:

        # 초기화
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(iterations):

            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
            print("[step: {}] loss: {}".format(i, step_loss))


        # Test step
        test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}), originalXY)
        realSale = minMaxDeNormalizer(testY[-1], originalXY)
#         listedLogPredict=test_predict[-1].tolist()
#     return [np.exp(y) for y in listedLogPredict]
    return np.square(test_predict[-1]).tolist()

def Bayseian(txs, forecastDay, unit):
    global mockForecastDictionary
    global realForecastDictionary

    if unit is 'day':
        if (len(txs) < 366):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)


    elif unit is 'week':
        if(len(txs)<53):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

    elif unit is 'month':
        if(len(txs)<12):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)


    #date = [d.strftime('%Y-%m-%d') for d in forecastProphetTable['ds']]
    return [np.exp(y) for y in forecastProphetTable['yhat'][-forecastDay:]]

def rmse(a,b):
    sum=0
    for i in range(len(a)):
        sum=sum+(a[i]-b[i])**2
    return np.sqrt(sum/len(a))

def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

def minMaxDeNormalizer(data, originalData):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier

def AlgorithmCompare(testY):
    global mockForecastDictionary
    nameOfBestAlgorithm = 'LSTM'
    minData = rmse(testY, mockForecastDictionary[nameOfBestAlgorithm])
    rms = 0
    for algorithm in mockForecastDictionary.keys():
        rms = rmse(testY, mockForecastDictionary[algorithm])
        if rms < minData:
            nameOfBestAlgorithm = algorithm
    print('testY is: ', testY)
    print('\n')
    print('LSTM testforecast :',mockForecastDictionary['LSTM'] , '\n@@@@@LSTM rmse: ',rmse(testY, mockForecastDictionary['LSTM']) )
    print('Bayseian testforecast :',mockForecastDictionary['Bayseian'], '\n@@@@@Bayseian rmse: ',rmse(testY, mockForecastDictionary['Bayseian']) ) 
    print('\n')
    print(nameOfBestAlgorithm, 'WON!!!!!!')
    return nameOfBestAlgorithm


In [2]:
# def LSTM(txs, forecastDay, features)
# def Bayseian(txs, forecastDay, unit)

In [9]:
columns=['ds','y']
df=pd.read_table('walMonth24.csv', sep=',',header=None,names=columns )
forecastDay=int(len(df)*0.2)
print(len(df), forecastDay)


24 4


In [10]:
df['y'][-forecastDay:]=[1]*forecastDay
rawArrayDatas=[[i for i in df['ds']],[i for i in df['y']]]

C:\Users\User\Anaconda3\envs\tensorflow35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
rawArrayDatas

[['2010-02-01',
  '2010-03-01',
  '2010-04-01',
  '2010-05-01',
  '2010-06-01',
  '2010-07-01',
  '2010-08-01',
  '2010-09-01',
  '2010-10-01',
  '2010-11-01',
  '2010-12-01',
  '2011-01-01',
  '2011-02-01',
  '2011-03-01',
  '2011-04-01',
  '2011-05-01',
  '2011-06-01',
  '2011-07-01',
  '2011-08-01',
  '2011-09-01',
  '2011-10-01',
  '2011-11-01',
  '2011-12-01',
  '2012-01-01'],
 [32990.769999999997,
  22809.285,
  30103.352000000003,
  16673.537499999999,
  16685.174999999999,
  16383.002,
  16144.702499999999,
  17978.317500000001,
  26928.906000000003,
  23040.349999999999,
  34796.775999999998,
  17286.647499999999,
  31440.657500000001,
  20705.834999999999,
  33011.389999999999,
  17062.93,
  15744.6425,
  15771.246000000001,
  14765.487499999999,
  17551.281999999999,
  1.0,
  1.0,
  1.0,
  1.0]]

In [12]:
answer=LearningModuleRunner(rawArrayDatas,0,forecastDay,'month') #0은 processId

[step: 0] loss: 0.24472439289093018
[step: 1] loss: 0.12649907171726227
[step: 2] loss: 0.05139520391821861
[step: 3] loss: 0.012810866348445415
[step: 4] loss: 0.003557896474376321
[step: 5] loss: 0.013666285201907158
[step: 6] loss: 0.029137030243873596
[step: 7] loss: 0.038231976330280304
[step: 8] loss: 0.038096074014902115
[step: 9] loss: 0.03156135603785515
[step: 10] loss: 0.0224484633654356
[step: 11] loss: 0.013684406876564026
[step: 12] loss: 0.006947314366698265
[step: 13] loss: 0.0028865919448435307
[step: 14] loss: 0.0014300041366368532
[step: 15] loss: 0.002036311663687229
[step: 16] loss: 0.00389351905323565
[step: 17] loss: 0.006115085911005735
[step: 18] loss: 0.007955591194331646
[step: 19] loss: 0.008974166586995125
[step: 20] loss: 0.009067922830581665
[step: 21] loss: 0.00838521495461464
[step: 22] loss: 0.007196047343313694
[step: 23] loss: 0.005785008426755667
[step: 24] loss: 0.004389102570712566
[step: 25] loss: 0.0031755699310451746
[step: 26] loss: 0.00224489

[step: 217] loss: 0.0012357558589428663
[step: 218] loss: 0.0012354813516139984
[step: 219] loss: 0.0012352027697488666
[step: 220] loss: 0.0012349230237305164
[step: 221] loss: 0.001234645489603281
[step: 222] loss: 0.0012343658600002527
[step: 223] loss: 0.0012340834364295006
[step: 224] loss: 0.0012338024098426104
[step: 225] loss: 0.0012335206847637892
[step: 226] loss: 0.0012332384940236807
[step: 227] loss: 0.0012329549062997103
[step: 228] loss: 0.0012326715514063835
[step: 229] loss: 0.0012323884293437004
[step: 230] loss: 0.0012321012327447534
[step: 231] loss: 0.0012318184599280357
[step: 232] loss: 0.0012315307976678014
[step: 233] loss: 0.0012312415055930614
[step: 234] loss: 0.001230957917869091
[step: 235] loss: 0.001230668742209673
[step: 236] loss: 0.0012303771218284965
[step: 237] loss: 0.001230088178999722
[step: 238] loss: 0.0012297978391870856
[step: 239] loss: 0.0012295072665438056
[step: 240] loss: 0.001229214365594089
[step: 241] loss: 0.0012289222795516253
[step

[step: 437] loss: 0.0011584318708628416
[step: 438] loss: 0.0011579979909583926
[step: 439] loss: 0.001157563179731369
[step: 440] loss: 0.0011571261566132307
[step: 441] loss: 0.00115669216029346
[step: 442] loss: 0.0011562567669898272
[step: 443] loss: 0.0011558164842426777
[step: 444] loss: 0.0011553775984793901
[step: 445] loss: 0.00115493917837739
[step: 446] loss: 0.001154496567323804
[step: 447] loss: 0.0011540558189153671
[step: 448] loss: 0.001153611927293241
[step: 449] loss: 0.0011531682685017586
[step: 450] loss: 0.0011527251917868853
[step: 451] loss: 0.0011522802524268627
[step: 452] loss: 0.0011518358951434493
[step: 453] loss: 0.0011513869976624846
[step: 454] loss: 0.0011509375181049109
[step: 455] loss: 0.0011504903668537736
[step: 456] loss: 0.0011500392574816942
[step: 457] loss: 0.0011495882645249367
[step: 458] loss: 0.0011491378536447883
[step: 459] loss: 0.0011486838338896632
[step: 460] loss: 0.0011482315603643656
[step: 461] loss: 0.0011477770749479532
[step: 

[step: 659] loss: 0.001031004823744297
[step: 660] loss: 0.0010302134323865175
[step: 661] loss: 0.001029420644044876
[step: 662] loss: 0.0010286231990903616
[step: 663] loss: 0.001027823076583445
[step: 664] loss: 0.0010270203929394484
[step: 665] loss: 0.0010262143332511187
[step: 666] loss: 0.001025402219966054
[step: 667] loss: 0.0010245905723422766
[step: 668] loss: 0.0010237740352749825
[step: 669] loss: 0.0010229558683931828
[step: 670] loss: 0.0010221336269751191
[step: 671] loss: 0.001021307660266757
[step: 672] loss: 0.0010204779682680964
[step: 673] loss: 0.0010196459479629993
[step: 674] loss: 0.0010188099695369601
[step: 675] loss: 0.0010179686360061169
[step: 676] loss: 0.00101712706964463
[step: 677] loss: 0.0010162802645936608
[step: 678] loss: 0.0010154314804822206
[step: 679] loss: 0.0010145781561732292
[step: 680] loss: 0.0010137201752513647
[step: 681] loss: 0.001012858934700489
[step: 682] loss: 0.001011995249427855
[step: 683] loss: 0.0010111266747117043
[step: 68

[step: 880] loss: 0.0007542454986833036
[step: 881] loss: 0.0007559215882793069
[step: 882] loss: 0.0007532855961471796
[step: 883] loss: 0.000749852042645216
[step: 884] loss: 0.0007506865658797324
[step: 885] loss: 0.000748421298339963
[step: 886] loss: 0.0007453711004927754
[step: 887] loss: 0.0007455243612639606
[step: 888] loss: 0.0007437010062858462
[step: 889] loss: 0.0007408509263768792
[step: 890] loss: 0.0007404139614664018
[step: 891] loss: 0.0007390093524008989
[step: 892] loss: 0.0007363441400229931
[step: 893] loss: 0.0007353475084528327
[step: 894] loss: 0.0007342554163187742
[step: 895] loss: 0.0007318667485378683
[step: 896] loss: 0.0007303573656827211
[step: 897] loss: 0.0007293744711205363
[step: 898] loss: 0.0007273794617503881
[step: 899] loss: 0.0007254896918311715
[step: 900] loss: 0.0007243636646308005
[step: 901] loss: 0.0007227860623970628
[step: 902] loss: 0.0007207826711237431
[step: 903] loss: 0.0007193080964498222
[step: 904] loss: 0.0007179768290370703
[s

[step: 126] loss: 0.0034017120487987995
[step: 127] loss: 0.003400502260774374
[step: 128] loss: 0.0033992871176451445
[step: 129] loss: 0.0033980391453951597
[step: 130] loss: 0.003396750194951892
[step: 131] loss: 0.003395422827452421
[step: 132] loss: 0.0033940779976546764
[step: 133] loss: 0.0033927266485989094
[step: 134] loss: 0.0033913799561560154
[step: 135] loss: 0.003390049561858177
[step: 136] loss: 0.00338873453438282
[step: 137] loss: 0.003387431614100933
[step: 138] loss: 0.003386132884770632
[step: 139] loss: 0.003384823678061366
[step: 140] loss: 0.003383506555110216
[step: 141] loss: 0.0033821621909737587
[step: 142] loss: 0.003380811307579279
[step: 143] loss: 0.003379451110959053
[step: 144] loss: 0.0033780885860323906
[step: 145] loss: 0.003376731416210532
[step: 146] loss: 0.0033753737807273865
[step: 147] loss: 0.0033740221988409758
[step: 148] loss: 0.0033726671244949102
[step: 149] loss: 0.003371313214302063
[step: 150] loss: 0.003369948361068964
[step: 151] los

[step: 375] loss: 0.002992551075294614
[step: 376] loss: 0.002990629058331251
[step: 377] loss: 0.002988707274198532
[step: 378] loss: 0.002986784791573882
[step: 379] loss: 0.0029848581179976463
[step: 380] loss: 0.002982932608574629
[step: 381] loss: 0.002981003141030669
[step: 382] loss: 0.0029790718108415604
[step: 383] loss: 0.002977140247821808
[step: 384] loss: 0.002975204959511757
[step: 385] loss: 0.0029732640832662582
[step: 386] loss: 0.002971323672682047
[step: 387] loss: 0.002969387685880065
[step: 388] loss: 0.0029674433171749115
[step: 389] loss: 0.0029654991813004017
[step: 390] loss: 0.0029635531827807426
[step: 391] loss: 0.002961604855954647
[step: 392] loss: 0.0029596579261124134
[step: 393] loss: 0.0029577049426734447
[step: 394] loss: 0.0029557514935731888
[step: 395] loss: 0.0029537982773035765
[step: 396] loss: 0.002951841102913022
[step: 397] loss: 0.0029498785734176636
[step: 398] loss: 0.0029479211661964655
[step: 399] loss: 0.0029459635261446238
[step: 400] 

[step: 624] loss: 0.0024584794882684946
[step: 625] loss: 0.0024561025202274323
[step: 626] loss: 0.002453723456710577
[step: 627] loss: 0.0024513406679034233
[step: 628] loss: 0.0024489564821124077
[step: 629] loss: 0.0024465746246278286
[step: 630] loss: 0.0024441878776997328
[step: 631] loss: 0.002441796939820051
[step: 632] loss: 0.0024394053034484386
[step: 633] loss: 0.0024370127357542515
[step: 634] loss: 0.0024346215650439262
[step: 635] loss: 0.002432223642244935
[step: 636] loss: 0.002429824322462082
[step: 637] loss: 0.002427423372864723
[step: 638] loss: 0.002425020094960928
[step: 639] loss: 0.0024226189125329256
[step: 640] loss: 0.0024202135391533375
[step: 641] loss: 0.0024178025778383017
[step: 642] loss: 0.0024153939448297024
[step: 643] loss: 0.0024129794910550117
[step: 644] loss: 0.0024105655029416084
[step: 645] loss: 0.002408148255199194
[step: 646] loss: 0.0024057317059487104
[step: 647] loss: 0.0024033072404563427
[step: 648] loss: 0.0024008839391171932
[step: 

[step: 876] loss: 0.0018809587927535176
[step: 877] loss: 0.0018786832224577665
[step: 878] loss: 0.0018773155752569437
[step: 879] loss: 0.0018749615410342813
[step: 880] loss: 0.0018729653675109148
[step: 881] loss: 0.0018714123871177435
[step: 882] loss: 0.001869112835265696
[step: 883] loss: 0.0018672436708584428
[step: 884] loss: 0.00186556251719594
[step: 885] loss: 0.0018633584259077907
[step: 886] loss: 0.0018615317530930042
[step: 887] loss: 0.0018597810994833708
[step: 888] loss: 0.0018576699076220393
[step: 889] loss: 0.00185584113933146
[step: 890] loss: 0.001854069298133254
[step: 891] loss: 0.001852030516602099
[step: 892] loss: 0.0018501896411180496
[step: 893] loss: 0.0018484167521819472
[step: 894] loss: 0.001846440602093935
[step: 895] loss: 0.0018445819150656462
[step: 896] loss: 0.0018428121693432331
[step: 897] loss: 0.001840893761254847
[step: 898] loss: 0.0018390242476016283
[step: 899] loss: 0.0018372524064034224
[step: 900] loss: 0.0018353909254074097
[step: 90

In [13]:
len(answer)

4

In [14]:
answer

[757618.93777588557, 761722.2857351097, 921883.73829621298, 329503.70969187602]

In [15]:
len(rawArrayDatas[1])

24